In [ ]:
!pip3 install --user \
    langchain==0.0.217 \
    pypdf \
    chromadb==0.3.26 \
    openai

In [ ]:
%pip install openai==0.28.1
%pip install langchain==0.0.330

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
import chromadb
import openai

In [ ]:
loader = PyPDFLoader("Hasan Rahman - Resume.pdf")
pages = loader.load_and_split()

In [ ]:
pages[0]

In [ ]:
import os

# Uncomment the following line to set the environment variable in the notebook
os.environ["OPENAI_API_KEY"] = ''

if os.getenv("OPENAI_API_KEY") is not None:
    print("OPENAI_API_KEY is ready")
    import openai
    openai.api_key = os.getenv("OPENAI_API_KEY")
else:
    print("OPENAI_API_KEY environment variable not found")

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 800,
    chunk_overlap  = 400,
    length_function = len,
)

chunks = text_splitter.split_documents(pages)

# Look at the first two chunks 
chunks[0:2]

In [ ]:
print(f'Number of documents: {len(pages)}')
print(f'Number of chunks: {len(chunks)}')

In [ ]:
embeddings = OpenAIEmbeddings()

In [ ]:
# set persist directory so the vector store is saved to disk
db = Chroma.from_documents(chunks, embeddings, persist_directory="./vectorstore")

In [ ]:
# vector store 
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k":5} # number of nearest neighbors to retrieve  
)


# q/a chain 
qa = RetrievalQA.from_chain_type(
    OpenAI(),
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=False
)

In [ ]:
qa({"query": "why is he a capable sales engineer. give me one sentence, thats it."})